# Booking.com Hotel Review NLP

Features by relationship

Hotel
- address
- hotel name
- lat
- long
- total_number_of_reviews
- additional_number_of_scoring
- average_score

reviewer
- review_nationality
- total_number_of_reviews_reviewer_has_given
- reviewer_score
- tags

review
- negative_review
- review_total_negative_word_counts
- negative_unique_word_count
- positive_review
- review_total_postive_word_counts
- positive_uniqu_word_count
- days_since_review

date
- month
- year
- day_name
- day
- quarter

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import re
import wrangle

pd.options.display.max_colwidth = None
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Acquire

- How do we aquire this from the webpage programatically?

In [2]:
hotel = pd.read_csv('Hotel_Reviews.csv')

In [3]:
hotel = wrangle.wrangle_hotel(hotel)

In [4]:
hotel.head(1)

,hotel_address,additional_number_of_scoring,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,month,year,day_name,day,quarter,negative_unique_word_count,positive_unique_word_count,location
review_date,,,,,,,,,,,,,,,,,,,,,,,,
2015-08-04,225 Edgware Road Westminster Borough London W2 1JU United Kingdom,1485,7.5,Hilton London Metropole,United Kingdom,First day the swimming pool was not opened therefore i could not access the facilities Finaly the receptionist fid not clarify that i would unable to move my car and did not stamp my car parking so i can come in and out Therefore i had to pay 50 for one day,54,6977,The hotel was clean and accessible,8,4,7.5,"[leisure trip, group, superior queen room, stayed 2 nights, submitted from a mobile device]",730,51.519569,-0.170521,August,2015,Tuesday,4,3,39,6,United Kingdom


# Wrangle

### Wrangle Function

In [5]:
# def wrangle_hotel(df):
#     '''
#     Wrangle Start
#     '''
    
#     # lower case column names
#     df.columns = [col.lower() for col in df]
    
#     # Set the review date as a datetime object then set it as the index
#     df.review_date = pd.to_datetime(df.review_date)
#     df = df.set_index('review_date').sort_index()
    
#     # Create columns for date types to groupby
#     df['month'] = df.index.month_name()
#     df['year'] = df.index.year
#     df['day_name'] = df.index.day_name()
#     df['day'] = df.index.day
#     df['quarter'] = df.index.quarter
    
#     # Unique word counts for positive and negative reviews
#     df['negative_unique_word_count'] = [len(set(nr.split())) for nr in df.negative_review]
#     df['positive_unique_word_count'] = [len(set(pr.split())) for pr in df.positive_review]
    
#     # remove day string and make int type
#     df.days_since_review = [row.split()[0] for row in df.days_since_review]
#     df.days_since_review = df.days_since_review.astype('int')
    
#     # Get Hotel Location
#     df['location'] = [' '.join(col.split()[-2:]) for col in df.hotel_address]
    
#     # Break out tags into groups
#     df.tags = [[tag.strip().lower() for tag in (tags.replace('"','').replace("'","")
#                                      .replace('[','') .replace(']','')
#                                      .split(','))] for tags in df.tags]
    
#     return df

In [6]:
# lower case column names
# hotel.columns = [col.lower() for col in hotel]

In [7]:
# # Set the review date as a datetime object then set it as the index
# hotel.review_date = pd.to_datetime(hotel.review_date)
# hotel = hotel.set_index('review_date').sort_index()
# # Create columns for date types to groupby
# hotel['month'] = hotel.index.month_name()
# hotel['year'] = hotel.index.year
# hotel['day_name'] = hotel.index.day_name()
# hotel['day'] = hotel.index.day
# hotel['quarter'] = hotel.index.quarter

In [8]:
# # Unique word count for first row
# #len(set(hotel.negative_review[0].split()))

# # Unique word counts for positive and negative reviews
# hotel['negative_unique_word_count'] = [len(set(nr.split())) for nr in hotel.negative_review]
# hotel['positive_unique_word_count'] = [len(set(pr.split())) for pr in hotel.positive_review]

In [9]:
# # remove day string and make int type
# hotel.days_since_review = [row.split()[0] for row in hotel.days_since_review]
# hotel.days_since_review = hotel.days_since_review.astype('int')

In [10]:
# # Get Hotel Location
# hotel['location'] = [' '.join(col.split()[-2:]) for col in hotel.hotel_address]

In [11]:
# # Break out tags into groups
# hotel.tags = [[tag.strip().lower() for tag in (tags.replace('"','').replace("'","")
#                                      .replace('[','') .replace(']','')
#                                      .split(','))] for tags in hotel.tags]

### Address

In [12]:
# We need to figure out a way to seperate out the address in to its parts.
# Street Address
# City
# Zip
# Country
# hotel.hotel_address.value_counts()

In [13]:
hotel.hotel_address[0].split()[-2:]

['United', 'Kingdom']

In [14]:
hotel.location.value_counts()

United Kingdom           262301
Barcelona Spain           60149
Paris France              59928
Amsterdam Netherlands     57214
Vienna Austria            38939
Milan Italy               37207
Name: location, dtype: int64

### Review Lengths

In [15]:
# Length of negative review
len(hotel.negative_review[0].split())

52

In [16]:
# Given length of negative review seems wrong and should be investigated further
hotel.review_total_negative_word_counts[0]

54

### Tags

In [17]:
pd.Series([len(tags.split(',')) for tags in hotel.tags]).value_counts().sort_index()

AttributeError: 'list' object has no attribute 'split'

In [ ]:
hotel.tags[0][0]

In [ ]:
# [tag for tag in hotel.tags if len(tag) == 6]
# [tag for tag in hotel.tags if len(tag) == 5]
# [tag for tag in hotel.tags if len(tag) == 4]
# [tag for tag in hotel.tags if len(tag) == 3]
# [tag for tag in hotel.tags if len(tag) == 2]
# [tag for tag in hotel.tags if len(tag) == 1]

In [ ]:
hotel.head(1)

# Explore

In [ ]:
hotel.describe()

In [ ]:
hotel.info()

## Univariate

### Mising Values

In [ ]:
# print nulls per row
[print(f'{col}:{hotel[col].isnull().sum()}') for col in  hotel.columns] # How to format better

### No valid review

In [ ]:
hotel.negative_review[2]

In [ ]:
(hotel.positive_review == 'No Positive').mean()

In [ ]:
(hotel.positive_review == ' Nothing').mean()

In [ ]:
(hotel.negative_review == 'No Negative').mean()

In [ ]:
(hotel.negative_review == ' Nothing').mean()

In [ ]:
# Unique Values per column
# for col in hotel.columns:
#     print(f'{col} Counts:{hotel[col].nunique()}')